# Packages

In [1]:
# -- Import packages

import pyclipper
from PIL import Image
import pandas as pd
import os
import glob as glob
import SimpleITK as sitk
import pathlib
import roifile
import numpy as np
from scipy.spatial import ConvexHull
import cv2 
import radiomics
from radiomics.featureextractor import RadiomicsFeatureExtractor
import spatial_efd
import traceback
import logging
from scipy.sparse import csr_matrix, find
import time
from typing import List, Tuple, Callable, Optional, Union, Dict, Any

logging.getLogger('radiomics').setLevel(logging.CRITICAL + 1)

%load_ext watermark

In [10]:
%watermark

from watermark import watermark
%watermark --iversions

Last updated: 2024-09-24T10:48:57.132457+02:00

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.19.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 141 Stepping 1, GenuineIntel
CPU cores   : 16
Architecture: 64bit

numpy      : 1.19.2
pandas     : 1.3.5
pyclipper  : 1.2.1
PIL        : 8.0.1
watermark  : 2.5.0
spatial_efd: 1.2.1
SimpleITK  : 2.0.2
roifile    : 2021.6.6
scipy      : 1.5.2
radiomics  : 2.2.0b1
logging    : 0.5.1.2
cv2        : 4.5.1



# Feature extraction functions

In [2]:
# -- Functions

def compute_polygon_area(vertices: List[Tuple[int, int]],
                        ) -> float:
    
    """
    Computes the area of a polygon using the Shoelace formula (also known as 
    Gauss's area formula), given its vertices.

    Parameters:
    -----------
    vertices : List[Tuple[int, int]]
        A list of tuples, where each tuple represents the (x, y) coordinates of 
        the polygon's vertices in order. The polygon is assumed to be closed, 
        meaning the last vertex implicitly connects to the first.

    Returns:
    --------
    float
        The area of the polygon. The area will always be a positive value.
    """
    
    n = len(vertices)
    area = 0.0
    for i in range(n):
        x1, y1 = vertices[i]
        x2, y2 = vertices[(i + 1) % n]
        area += x1 * y2 - x2 * y1
        
    return abs(area) / 2.0

def compute_intensity_features_from_marker(stack_sitk: sitk.SimpleITK.Image, 
                                           mask_sitk: sitk.SimpleITK.Image,
                                           channel: int,
                                           feature_selection: Dict[str, List[str]],
                                           params : Dict,
                                           label: int,
                                           x_spacing: Optional[int] = 1,
                                           y_spacing: Optional[int] = 1
                                          ) -> Dict[str, Any]:
    
    """
    Computes intensity-based features from a given image stack and a corresponding mask for a specific label.

    Parameters:
    -----------
    stack_sitk : sitk.Image
        The input image stack (in SimpleITK format) from which intensity features are computed. 
        It may contain multiple channels.
    
    mask_sitk : sitk.Image
        A mask image (in SimpleITK format) used to isolate the region of interest (ROI) 
        where the intensity features will be computed.
    
    channel : int
        The channel index in the image stack from which the intensity features should be calculated.
    
    feature_selection : Dict[str, List[str]]
        A dictionary specifying the types of features to compute. The keys are feature categories 
        (e.g., "intensity", "texture") and the values are lists of feature names to calculate.
    
    params : Dict
        A dictionary of additional parameters or settings needed for feature computation. 
        This may include preprocessing settings, normalization factors, or other configurations.
    
    label : int
        The label in the mask that corresponds to the region of interest for which the features 
        are being calculated.
    
    x_spacing : Optional[int], default=1
        The pixel spacing in the x-direction, used for scaling the feature computations if 
        physical spacing needs to be taken into account.
    
    y_spacing : Optional[int], default=1
        The pixel spacing in the y-direction, similar to `x_spacing`, used to adjust for 
        physical spacing during feature computation.
    
    Returns:
    --------
    Dict[str, Any]
        A dictionary of computed intensity features. The keys are the feature names (as specified 
        by `feature_selection`), and the values are the corresponding computed values.
    """
    
    if feature_selection:
        radiomics_extractor = RadiomicsFeatureExtractor(verbose=False, preCrop=True, additionalInfo=False)
        radiomics_extractor.disableAllFeatures()
        radiomics_extractor.enableFeaturesByName(**feature_selection)
        features = radiomics_extractor.computeFeatures(stack_sitk[:,:,channel],
                                                       mask_sitk, 
                                                       imageTypeName='radiomics', 
                                                       label=label,
                                                       **params) 
        
    else:
        features = {}
    
    return features


def compute_intensity_features_from_cell(markers_config: Dict,
                                         stack: sitk.SimpleITK.Image,
                                         nuclear_mask: sitk.SimpleITK.Image,
                                         cyto_mask: sitk.SimpleITK.Image,
                                         perinuclear_mask: sitk.SimpleITK.Image,
                                         cell_id: int,
                                         x_spacing: Optional[int] = 1,
                                         y_spacing: Optional[int] = 1
                                        ) -> Dict[str, Any]:
    
    """
    Computes intensity features for a specific cell, given its associated masks and marker configuration.

    Parameters:
    -----------
    markers_config : Dict
        A configuration dictionary specifying the markers to analyze. Each key is the marker name, 
        and the value is a dictionary with details including:
        - 'channel': int, the index of the channel corresponding to the marker.
        - 'features': List[str], the specific features to compute for this marker.
        - 'params': Dict, any additional parameters needed for feature computation.
        - 'compartment': str, the cell compartment ('nuclear', 'cyto', or 'perinuclear') to analyze.
    
    stack : sitk.Image
        The image stack (in SimpleITK format) containing all the channels used for intensity feature computation.
    
    nuclear_mask : sitk.Image
        A binary mask that isolates the nuclear compartment of the cell.
    
    cyto_mask : sitk.Image
        A binary mask that isolates the cytoplasmic compartment of the cell.
    
    perinuclear_mask : sitk.Image
        A binary mask that isolates the perinuclear region of the cell.
    
    cell_id : int
        The ID of the cell (within the masks) for which the features should be computed.
    
    x_spacing : Optional[int], default=1
        The pixel spacing in the x-direction, used for scaling the feature computations.
    
    y_spacing : Optional[int], default=1
        The pixel spacing in the y-direction, used for scaling the feature computations.
    
    Returns:
    --------
    Dict[str, Any]
        A dictionary of computed intensity features for the given cell. The keys are the marker names concatenated 
        with the feature names, and the values are the computed feature values.

    Notes:
    ------
    - The function computes features separately for different cell compartments (nuclear, cytoplasmic, perinuclear), 
      as specified in the `markers_config` dictionary.
    - It uses the `compute_intensity_features_from_marker` function to compute features for each marker.
    
    Example:
    --------
    markers_config = {
        "marker1": {
            "channel": 0,
            "features": ["mean", "std_dev"],
            "params": {"normalize": True},
            "compartment": "nuclear"
        },
        "marker2": {
            "channel": 1,
            "features": ["min", "max"],
            "params": {"normalize": False},
            "compartment": "cyto"
        }
    }
    """
    
    mask_dict = {'nuclear': nuclear_mask,
                 'cyto': cyto_mask,
                 'perinuclear': perinuclear_mask}
    
    cell_features = {}
    
    for marker in list(markers_config.keys()):

        channel = markers_config[marker]['channel']
        feature_selection = markers_config[marker]['features']
        params = markers_config[marker]['params']
        compartment = markers_config[marker]['compartment']

        marker_features = compute_intensity_features_from_marker(stack, 
                                                   mask_dict[compartment],
                                                   channel,
                                                   feature_selection,
                                                   params,
                                                   label=cell_id,
                                                   x_spacing=1,
                                                   y_spacing=1)

        marker_features = {marker + key.replace('radiomics_', '_'): value.reshape((1))[0] 
                           for key, value in zip(marker_features.keys(), marker_features.values())}

        cell_features.update(marker_features)
        

    return cell_features
        
def compute_shape_features(stack_crop_sitk: sitk.SimpleITK.Image, 
                           mask_crop_sitk: sitk.SimpleITK.Image, 
                           polygon_tuple: List[Tuple[int, int]], 
                           label: int, 
                           seg_channel: Optional[int] = 0,
                           harmonics: Optional[int] = 25
                          ) -> Dict[str, Any]:

    """
    Computes various shape features from a segmented region within an image stack, using both geometric
    properties and elliptical Fourier descriptors (EFD) for more detailed shape analysis.

    Parameters:
    -----------
    stack_crop_sitk : sitk.Image
        The cropped image stack (in SimpleITK format) from which the shape features will be computed. 
        This contains the segmented regions of interest.
    
    mask_crop_sitk : sitk.Image
        A mask image that isolates the region of interest (ROI) from which shape features will be extracted.
    
    polygon_tuple : List[Tuple[int, int]]
        A list of tuples, where each tuple contains the (x, y) coordinates of the polygon's vertices 
        that define the boundary of the region to be analyzed.
    
    label : int
        The label index within the mask that corresponds to the region of interest (e.g., the segmented region).
    
    seg_channel : Optional[int], default=0
        The channel of the image stack from which to extract the shape features. 
    
    harmonics : Optional[int], default=25
        The number of harmonic components to use for elliptical Fourier descriptor (EFD) analysis. 
        Higher values provide more detailed shape descriptions.

    Returns:
    --------
    Dict[str, Any]
        A dictionary of computed shape features, including:
        - 'major_axis': The length of the major axis of the region.
        - 'minor_axis': The length of the minor axis of the region.
        - 'perimeter': The perimeter of the region.
        - 'convex_area': The area of the convex hull surrounding the region.
        - 'area': The area of the region.
        - 'circularity': The circularity of the region (based on the area and perimeter).
        - 'roundness': The roundness of the region (based on the area and major axis).
        - 'solidity': The solidity of the region (area/convex area).
        - 'aspect_ratio': The ratio of the major to minor axis.
        - 'compactness': The compactness of the region (perimeter^2 / 4π * area).
        - 'efc_ratio': The ratio between EFD coefficients.
        - EFD coefficients for each harmonic, named as 'efd_a{i}b{j}'.
    """
    
    # perimeter, major_axis, minor_axis (computed using the pyradiomics library)
    radiomics_shape_features = radiomics.shape2D.RadiomicsShape2D(stack_crop_sitk[seg_channel,::], 
                                                                  mask_crop_sitk,
                                                                  label=label)    
    
    
    perimeter = radiomics_shape_features.getPerimeterFeatureValue()   
    major_axis = radiomics_shape_features.getMajorAxisLengthFeatureValue()
    minor_axis = radiomics_shape_features.getMinorAxisLengthFeatureValue()
    
    # area
    area = compute_polygon_area(polygon_tuple)
    
    # convex area, solidity, compactness
    hull = ConvexHull(np.array(polygon_tuple), incremental=True)
    convex_area = hull.volume
    solidity = area / convex_area 
    compactness = perimeter**2 / (4*np.pi*area)
    
    # aspect ratio, circularity, roundness
    aspect_ratio = major_axis / minor_axis
    circularity = 4*np.pi*(area/perimeter**2)
    roundness = 4*(area/(np.pi*major_axis**2))
    
    # EFD coefficients and EFC ratio 
    x_polygon = [point[0] for point in polygon_tuple]
    y_polygon = [point[1] for point in polygon_tuple]
    
    EFDcoeffs = spatial_efd.CalculateEFD(x_polygon, y_polygon, harmonics)
    EFDcoeffs, rotation = spatial_efd.normalize_efd(EFDcoeffs, size_invariant = True)
    
    MinorAxisArray = []
    MajorAxisArray = []
    
    for i in range(len(EFDcoeffs)):
        MinorAxis = (EFDcoeffs[i][0]**2 + EFDcoeffs[i][2]**2)**(1/2)
        MajorAxis = (EFDcoeffs[i][1]**2 + EFDcoeffs[i][3]**2)**(1/2)
        MinorAxisArray.append(MinorAxis)
        MajorAxisArray.append(MajorAxis)          

    efc_ratio = (MinorAxisArray[0] + MajorAxisArray[0])/( sum(MinorAxisArray[1:]) + sum(MajorAxisArray[1:]))
    
    dict_efd = {}
    for i in range(harmonics):
        dict_efd.update({f'efd_a{i}b{j}': EFDcoeffs[i,j] for j in range(4)})

    
    # Output
    shape_features =   {
                        'major_axis': major_axis,
                        'minor_axis': minor_axis,
                        'perimeter': perimeter,
                        'convex_area':convex_area,
                        'area': area,
                        'circularity': circularity,
                        'roundness': roundness,
                        'solidity': solidity,
                        'aspect_ratio': aspect_ratio,
                        'compactness': compactness,                              
                        'efc_ratio': efc_ratio
    }
    
    shape_features.update(dict_efd)
    
    return shape_features


def convert_mask_to_polygon(mask: np.ndarray,
                            label: int
                           ) -> Tuple[Tuple[int, int]]:
    
    """
    Converts a labeled region in a binary mask into a polygon by extracting its contour points.

    Parameters:
    -----------
    mask : np.ndarray
        A 2D NumPy array representing the mask, where different regions are labeled with integer values.
    
    label : int
        The specific label in the mask corresponding to the region of interest whose contour will be extracted.

    Returns:
    --------
    Tuple[Tuple[int, int]]
        A tuple of (x, y) coordinates representing the points of the polygon that outlines the region of interest.
        The points are extracted from the external contour of the region.

    Notes:
    ------
    - This function uses OpenCV to find the external contour of the region in the mask labeled with the given `label`.
    - It assumes that the mask is a 2D image with labeled regions.
    - The contour with the longest perimeter is selected if there are multiple external contours.

    Example:
    --------
    >>> mask = np.array([[0, 0, 0, 0],
                         [0, 1, 1, 0],
                         [0, 1, 1, 0],
                         [0, 0, 0, 0]])
    >>> polygon = convert_mask_to_polygon(mask, label=1)
    >>> print(polygon)
    ((1, 1), (1, 2), (2, 2), (2, 1))

    """
 
    gray = mask == label
    gray = gray.astype(np.uint8)*255
    cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2]
    argmax = np.argmax([len(cnts[i]) for i in range(len(cnts))])
    
    polygon_tuple = tuple( [ (point[0][0],
                              point[0][1]) for point in cnts[argmax] ] ) 
    
    
    return polygon_tuple

def get_object_coordinates(image_array: np.ndarray
                          ) -> Dict[str, np.ndarray]:
    
    """
    Extracts the coordinates of all unique objects in a labeled image array.

    Parameters:
    -----------
    image_array : np.ndarray
        A 2D or 3D NumPy array where each unique integer value represents a distinct object, 
        and the background is assumed to be labeled as 0.

    Returns:
    --------
    Dict[str, np.ndarray]
        A dictionary where the keys are the unique object labels (excluding the background, labeled as 0), 
        and the values are NumPy arrays containing the coordinates of the pixels or voxels that make up 
        each object. Each NumPy array has shape (N, D), where N is the number of pixels/voxels belonging 
        to the object, and D is the dimensionality of the image (e.g., 2 for 2D images or 3 for 3D images).

    Notes:
    ------
    - This function assumes that the background of the image is labeled as 0, and only non-zero objects 
      are included in the output.
    - The function works for both 2D and 3D images.
    - The coordinates are returned in (row, col) format for 2D images, and (z, row, col) for 3D images.

    Example:
    --------
    >>> image_array = np.array([[0, 0, 1, 1],
                                [0, 2, 2, 0],
                                [0, 2, 2, 0]])
    >>> get_object_coordinates(image_array)
    {1: array([[0, 2],
               [0, 3]]),
     2: array([[1, 1],
               [1, 2],
               [2, 1],
               [2, 2]])}
    """
    
    unique_objects = np.unique(image_array)
    object_coordinates = {}

    for obj in unique_objects:
        if obj != 0:  # Assuming 0 is the background value
            coords = np.argwhere(image_array == obj)
            object_coordinates[obj] = coords

    return object_coordinates

def replace_labels_with_zeros(array: np.ndarray, 
                              labels_to_replace: List[int]):
    
    """
    Replaces specific labels in a NumPy array with zeros.

    Parameters:
    -----------
    array : np.ndarray
        A NumPy array where each unique integer value represents a label (e.g., in a labeled image).
    
    labels_to_replace : List[int]
        A list of labels (integers) that should be replaced with 0 in the array.

    Returns:
    --------
    np.ndarray
        The modified array with the specified labels replaced by zeros. The shape of the array remains unchanged.

    Notes:
    ------
    - The function performs an in-place replacement of the labels in the array.
    - If any of the labels in `labels_to_replace` are not present in the array, the function simply skips those labels.
    - This function can be used to remove specific regions or objects in labeled images by replacing them with the background (0).

    Example:
    --------
    >>> array = np.array([[1, 2, 3],
                          [4, 5, 6],
                          [7, 8, 9]])
    >>> labels_to_replace = [2, 5, 9]
    >>> replace_labels_with_zeros(array, labels_to_replace)
    array([[1, 0, 3],
           [4, 0, 6],
           [7, 8, 0]])
    """

    for label in labels_to_replace:
        array[array == label] = 0
    return array

def is_border_object(image_array: np.ndarray, 
                     coords: np.ndarray,
                     ) -> bool:
    
    """
    Determines whether an object, defined by its coordinates, touches the border of an image.

    Parameters:
    -----------
    image_array : np.ndarray
        A 2D NumPy array representing the image or label mask. The dimensions of the array are used to 
        define the border region.
    
    coords : np.ndarray
        A NumPy array containing the coordinates of the object in the format (row, col). Each row of 
        the array represents the position of one pixel/voxel of the object in the image.

    Returns:
    --------
    bool
        Returns `True` if any part of the object touches the border of the image, `False` otherwise.
    """

    
    border_rows = np.concatenate( (np.arange(0,5), 
                                   np.arange(image_array.shape[0]-5,image_array.shape[0]-1) ), 
                                 axis=0)
    border_cols = np.concatenate( (np.arange(0,5), 
                                   np.arange(image_array.shape[1]-5,image_array.shape[1]-1) ),
                                 axis=0)
    
    for coord in coords:
        if coord[0] in border_rows or coord[1] in border_cols:
            return True
        
    return False

def remove_border_objects(mask: np.ndarray) -> np.ndarray:
    
    """
    Removes objects that touch the border of a labeled image mask by setting their labels to zero.

    Parameters:
    -----------
    mask : np.ndarray
        A 2D or 3D NumPy array where each unique integer value represents a labeled object, 
        and the background is assumed to be labeled as 0.

    Returns:
    --------
    np.ndarray
        The modified mask with all objects that touch the image border removed (i.e., their labels 
        are replaced by 0). The shape of the mask remains the same.
    """

    object_coordinates = get_object_coordinates(mask)
    object_centers = {obj: coords.mean(axis=0) for obj, coords in object_coordinates.items()}
    border_labels = []

    for obj, center in object_centers.items():
        border_object = is_border_object(mask, object_coordinates[obj])
        if border_object == True:
            border_labels.append(obj)

    mask = replace_labels_with_zeros(mask, border_labels)
    
    return mask

def generate_mask_crop(mask: np.ndarray, 
                       mask_csr: csr_matrix, 
                       label: int, 
                       delta: Optional[int] = 5 
                      ) -> Tuple[List[Tuple], np.ndarray]:
    
    """
    Generates a cropped version of a labeled mask around the region of a specific label.

    Parameters:
    -----------
    mask : np.ndarray
        A 2D NumPy array representing the labeled mask. Each unique integer value represents 
        a different object, and the background is assumed to be labeled as 0.
    
    mask_csr : csr_matrix
        A compressed sparse row (CSR) matrix version of the mask, used to efficiently locate the 
        positions of the object with the specified label.
    
    label : int
        The label (integer) of the object for which to generate the crop.
    
    delta : Optional[int], default=5
        A padding value specifying how many pixels to include around the bounding box of the object 
        in both x and y directions when creating the crop.

    Returns:
    --------
    Tuple[List[Tuple[int, int]], np.ndarray]
        A tuple containing:
        1. `crop_coordinates`: A list of two tuples, where the first tuple represents the (x_min, x_max) 
        coordinates and the second tuple represents the (y_min, y_max) coordinates of the cropped region.
        2. `mask_crop`: A 2D NumPy array representing the cropped mask around the specified object, 
        including the specified padding (`delta`).
    """
    
    x_cell, y_cell, values = find(mask_csr == label)
    
    x_min, x_max = x_cell.min(), x_cell.max()
    y_min, y_max = y_cell.min(), y_cell.max()

    crop_coordinates = [(x_min-delta, x_max+delta), (y_min-delta, y_max+delta)]
    
    mask_crop = mask[x_min-delta:x_max+delta, y_min-delta:y_max+delta]

    return crop_coordinates, mask_crop



def generate_stack_crop(stack_sitk: sitk.SimpleITK.Image, 
                        crop_coordinates: List[Tuple]
                       ) -> sitk.SimpleITK.Image:
    
    """
    Generates a cropped version of a SimpleITK image stack based on the provided crop coordinates.

    Parameters:
    -----------
    stack_sitk : sitk.Image
        A SimpleITK image stack from which a region will be cropped.
    
    crop_coordinates : List[Tuple[int, int]]
        A list of two tuples specifying the (x_min, x_max) and (y_min, y_max) coordinates for cropping. 
        The first tuple contains the y-coordinates, and the second tuple contains the x-coordinates 
        that define the region to be cropped.

    Returns:
    --------
    sitk.Image
        A cropped SimpleITK image stack extracted from the specified coordinates.
    """
    
    x_min, x_max = crop_coordinates[1]
    y_min, y_max = crop_coordinates[0]
    
    stack_crop_sitk = stack_sitk[x_min:x_max, y_min:y_max]
    
    return stack_crop_sitk

def generate_ring_mask_from_polygon(mask_size: Tuple,
                                    polygon_tuple_inner: List[Tuple[int, int]],
                                    polygon_tuple_outer: List[Tuple[int, int]],
                                    label: int
                                   ) -> np.ndarray:
    
    """
    Generates a ring-shaped mask by filling the area between two polygons: an inner polygon and an outer polygon.

    Parameters:
    -----------
    mask_size : Tuple[int, int]
        A tuple representing the size of the mask in (width, height) format.
    
    polygon_tuple_inner : List[Tuple[int, int]]
        A list of (x, y) coordinates representing the inner polygon, which defines the hole of the ring.
    
    polygon_tuple_outer : List[Tuple[int, int]]
        A list of (x, y) coordinates representing the outer polygon, which defines the boundary of the ring.

    label : int
        The label value to assign to the ring region in the mask.

    Returns:
    --------
    np.ndarray
        A 2D NumPy array (mask) of the specified size where the area between the inner and outer polygons is 
        filled with the provided `label`, and the rest is filled with zeros.
    """

    ring_mask = np.zeros((mask_size[1], mask_size[0]), dtype=np.int16)

    if polygon_tuple_inner and polygon_tuple_outer:

        inner_array = np.array([polygon_tuple_inner], 'int32') 
        outer_array = np.array([polygon_tuple_outer], 'int32')
        cv2.fillPoly(ring_mask, outer_array.astype(int), int(label))
        cv2.fillPoly(ring_mask, inner_array.astype(int), 0)
        
    return ring_mask

def generate_cyto_mask(polygon_tuple: List[Tuple[int, int]],
                       offset: int,
                       mask_size: Tuple[int,int],
                       label: int,
                       x_spacing: Optional[int] = 1,
                       y_spacing: Optional[int] = 1,
                      ) -> sitk.SimpleITK.Image:
    
    """
    Generates a cytoplasmic mask by creating an offset polygon around a given cell boundary and producing a ring mask 
    between the original cell boundary and the offset boundary.

    Parameters:
    -----------
    polygon_tuple : List[Tuple[int, int]]
        A list of (x, y) coordinates representing the original polygon, which typically defines the cell's boundary.
    
    offset : int
        The offset value used to expand the polygon outward, which defines the cytoplasmic region surrounding the cell.
    
    mask_size : Tuple[int, int]
        The size of the output mask in (width, height) format.
    
    label : int
        The label value to assign to the cytoplasmic mask region in the output mask.
    
    x_spacing : Optional[int], default=1
        The spacing in the x-direction, which will be set in the resulting SimpleITK image.
    
    y_spacing : Optional[int], default=1
        The spacing in the y-direction, which will be set in the resulting SimpleITK image.

    Returns:
    --------
    sitk.Image
        A SimpleITK image representing the cytoplasmic mask, where the region between the original polygon and 
        the offset polygon is filled with the provided `label`.
    """
    
    pco = pyclipper.PyclipperOffset()
    pco.AddPath(polygon_tuple, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
    polygon_offset = pco.Execute(offset)
    
    polygon_offset = tuple( [(point[0], point[1]) for point in polygon_offset[0]] )

    cyto_mask = generate_ring_mask_from_polygon(mask_size,
                                                polygon_tuple,
                                                polygon_offset,
                                                label
                                               )

    cyto_mask_sitk = sitk.GetImageFromArray(cyto_mask)
    cyto_mask_sitk.SetSpacing([x_spacing, y_spacing]) 
    
    return cyto_mask_sitk


def generate_perinuclear_mask(polygon_tuple: List[Tuple[int, int]],
                              inner_offset: int,
                              outer_offset: int,
                              mask_size: Tuple[int,int],
                              label: int,
                              x_spacing: Optional[int] = 1,
                              y_spacing: Optional[int] = 1,
                              ) -> sitk.SimpleITK.Image:
    
    """
    Generates a perinuclear mask by creating two offset polygons (inner and outer) around a given cell boundary, 
    and producing a ring mask between the two polygons.

    Parameters:
    -----------
    polygon_tuple : List[Tuple[int, int]]
        A list of (x, y) coordinates representing the original polygon, typically the cell's boundary.
    
    inner_offset : int
        The offset value used to create the inner polygon, defining the boundary closer to the cell nucleus.
    
    outer_offset : int
        The offset value used to create the outer polygon, defining the boundary further from the nucleus.
    
    mask_size : Tuple[int, int]
        The size of the output mask in (width, height) format.
    
    label : int
        The label value to assign to the perinuclear mask region in the output mask.
    
    x_spacing : Optional[int], default=1
        The spacing in the x-direction, which will be set in the resulting SimpleITK image.
    
    y_spacing : Optional[int], default=1
        The spacing in the y-direction, which will be set in the resulting SimpleITK image.

    Returns:
    --------
    sitk.Image
        A SimpleITK image representing the perinuclear mask, where the region between the inner and outer 
        offset polygons is filled with the provided `label`.
    """

    pco = pyclipper.PyclipperOffset()
    pco.AddPath(polygon_tuple, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
    polygon_inner = pco.Execute(inner_offset)
    polygon_outer = pco.Execute(outer_offset)
    
    polygon_inner = tuple( [(point[0], point[1]) for point in polygon_inner[0] ] )
    polygon_outer = tuple( [(point[0], point[1]) for point in polygon_outer[0] ] )


    perinuclear_mask = generate_ring_mask_from_polygon(mask_size,
                                                       polygon_inner,
                                                       polygon_outer,
                                                       label
                                                      )
    
    perinuclear_mask_sitk = sitk.GetImageFromArray(perinuclear_mask)
    perinuclear_mask_sitk.SetSpacing([x_spacing, y_spacing]) 
    
    return perinuclear_mask_sitk

def compute_cell_orientation(mask_csr: csr_matrix, 
                             label: int
                            ) -> np.ndarray:
    
    """
    Computes the orientation of a cell labeled in a sparse mask using eigenvalue decomposition on the 
    covariance matrix of the cell's coordinates.

    Parameters:
    -----------
    mask_csr : csr_matrix
        A compressed sparse row (CSR) matrix representation of the mask, where each non-zero element represents 
        a labeled object.
    
    label : int
        The label of the cell for which to compute the orientation.

    Returns:
    --------
    np.ndarray
        A 1D NumPy array representing the orientation vector of the cell, which corresponds to the eigenvector 
        associated with the smallest eigenvalue of the covariance matrix of the cell's coordinates. This vector 
        represents the principal axis of the cell's orientation.
    """
    
    x_cell, y_cell, values = find(mask_csr == label)
    coordinates = np.array([[x,y] for x,y in zip(x_cell, y_cell)]).astype('float64')
    
    coordinates -= np.mean(coordinates, axis=0)  # Centered at 0
    covariance = np.dot(coordinates.T.copy(), coordinates)
    eigenValues, eigenVectors = np.linalg.eig(covariance)  
    
    orientation_vector = eigenVectors[np.argmin(eigenValues),:]
    
    return orientation_vector

def compute_single_cell_features(img_path: str, 
                                mask_dir: str, 
                                output_dir: str, 
                                markers_config: Dict[str, Dict], 
                                features: List[str], 
                                remove_border: bool = True, 
                                out_format: str = 'csv', 
                                cyto_offset: int = 2, 
                                in_offset: int = -1, 
                                out_offset: int = 1, 
                                delta: int = 5, 
                                x_spacing: float = 1.0, 
                                y_spacing: float = 1.0
                            ):

    """
    Computes single-cell features from an image stack and its corresponding segmentation mask. 
    The features include shape, intensity, and texture information for the nucleus, cytoplasm, and perinuclear regions.

    Parameters:
    -----------
    img_path : str
        Path to the image stack (e.g., `.tif` file).
    
    mask_dir : str
        Directory containing the segmentation mask for the image stack (mask filename should match the image filename).
    
    output_dir : str
        Directory where the feature data will be saved.
    
    markers_config : Dict[str, Dict]
        A dictionary specifying markers and their respective channels and compartments. Each marker config should include:
        - 'channel': The index of the image stack channel corresponding to the marker.
        - 'features': List of intensity/texture features to compute.
        - 'compartment': The cellular compartment associated with the marker (e.g., 'nuclear', 'cyto', 'perinuclear').

    features : List[str]
        A list of features to compute. Options include 'intensity', 'shape', etc.

    remove_border : bool, default=True
        If `True`, cells at the image border are excluded from the analysis.
    
    out_format : str, default='csv'
        The output format for the feature data. Options are 'csv' or 'h5'.
    
    cyto_offset : int, default=2
        Offset used for generating the cytoplasm mask, defining the distance around the nucleus.

    in_offset : int, default=-1
        Offset used for generating the inner perinuclear boundary (shrinkage).

    out_offset : int, default=1
        Offset used for generating the outer perinuclear boundary (expansion).

    delta : int, default=5
        Padding value for cropping around the cell to reduce computational time.

    x_spacing : float, default=1.0
        Pixel spacing in the x-direction for the image.

    y_spacing : float, default=1.0
        Pixel spacing in the y-direction for the image.
    """
    
    compartments = list(set([markers_config[m]['compartment'] for m in list(markers_config.keys())] ))
    
    image_name = pathlib.PurePath(img_path).name
    
    if out_format == 'h5':
        out_path = os.path.join(output_dir, image_name.replace('.tif', '_features.h5'))
        out_files = glob.glob(output_dir + '/*.h5')

    if out_format == 'csv':
        out_path = os.path.join(output_dir, image_name.replace('.tif', '_features.csv'))
        out_files = glob.glob(output_dir + '/*.csv')
  

    if out_path not in out_files:
    
        # Open segmentation mask
        mask_name = image_name.replace('.tif', '_mask.png')
        mask_path = os.path.join(mask_dir, mask_name)
        mask = np.array(Image.open(mask_path))

        # Read image
        sitk.ProcessObject_SetGlobalWarningDisplay(False)
        file_reader = sitk.ImageFileReader()
        file_reader.SetFileName(img_path) 
        file_reader.ReadImageInformation()
        image_size = file_reader.GetSize()
        stack_sitk = file_reader.Execute() 

        # Remove cells located at the image periphery
        if remove_border:
            mask = remove_border_objects(mask)
        
        # Convert mask to a sparse matrix
        mask_csr = csr_matrix(mask)

        # Get cell ids from mask
        cell_ids = np.unique(mask)
        cell_ids = [x for x in cell_ids if x != 0] # remove background id
        print(f'{len(cell_ids)} cells detected')
        
        data = []
        for index, cell_id in enumerate(cell_ids):

            try:
                
                crop_coordinates, mask_crop = generate_mask_crop(mask, mask_csr, cell_id, delta=delta)
                
                # Crop mask around the cell to reduce computational time 
                mask_crop_sitk = sitk.GetImageFromArray(mask_crop)
                mask_crop_sitk.SetSpacing([x_spacing, y_spacing])   
                mask_crop_size = mask_crop_sitk.GetSize()
                    
                # Compute cell orientation
                orientation_vector = compute_cell_orientation(mask_csr, label=cell_id)

                # Convert mask to polygon to compute shape parameters and generate cyto and perinuclear masks
                polygon_tuple = convert_mask_to_polygon(mask_crop, cell_id) 

                # Crop the image stack around the cell to reduce computational time 
                stack_crop_sitk = generate_stack_crop(stack_sitk, crop_coordinates)
                crop_size = stack_crop_sitk.GetSize()
                
                
                # Generate a cytoplasm mask if requested
                if 'cyto' in compartments:
                    cyto_mask_sitk = generate_cyto_mask(polygon_tuple, 
                                                        offset=cyto_offset, 
                                                        mask_size=mask_crop_size,
                                                        label=cell_id,
                                                        x_spacing=x_spacing,
                                                        y_spacing=y_spacing
                                                        )
                else:
                    cyto_mask_sitk = None

                # Generate a perinuclear mask if requested
                if 'perinuclear' in compartments:
                    perinuclear_mask_sitk = generate_perinuclear_mask(polygon_tuple,
                                                                        inner_offset=in_offset,
                                                                        outer_offset=out_offset,
                                                                        mask_size=mask_crop_size,
                                                                        label=cell_id,
                                                                        x_spacing=x_spacing,
                                                                        y_spacing=y_spacing
                                                                        )
                    
                else:
                    perinuclear_mask_sitk = None



                # Get cell centroid (TEMPORARY)
                x_cell, y_cell, values = find(mask_csr == cell_id)
                x_centroid = x_cell.mean()
                y_centroid = y_cell.mean()
                
                # Add features to the dataframe
                feature_dict = {}
                feature_dict.update({
                                'image_name' : image_name.replace('.tif', ''),
                                'cell_id' : cell_id,
                                'x_centroid': x_centroid,
                                'y_centroid': y_centroid,
                                'x_orientation': orientation_vector[0],
                                'y_orientation': orientation_vector[1]
                                })    

                
                # Compute intensity and texture features

                if 'intensity' in features:

                    intensity_features = compute_intensity_features_from_cell(markers_config,
                                                                                stack=stack_crop_sitk,
                                                                                nuclear_mask=mask_crop_sitk,
                                                                                cyto_mask=cyto_mask_sitk,
                                                                                perinuclear_mask=perinuclear_mask_sitk,
                                                                                cell_id=int(cell_id),
                                                                                x_spacing=x_spacing,
                                                                                y_spacing=y_spacing)

                    feature_dict.update(intensity_features) 
                    
                # print(feature_dict)
                # Compute shape features
                if 'shape' in features:
                    shape_features = compute_shape_features(stack_crop_sitk, mask_crop_sitk, polygon_tuple, cell_id)

                    feature_dict.update(shape_features)


                data.append(feature_dict)

            except:
                pass

        data = pd.DataFrame.from_dict(data)

    #    Export data
        if out_format == 'h5':
            data.to_hdf(out_path, key='df', mode='w')

        if out_format == 'csv':
            data.to_csv(out_path, index=False)

# Configuration

In [13]:
# -- Configure script

DIRECTORY = r'C:\Users\fbertil\CRC_slides_restain\Slide 3\K10985-13\stain0'
MASK_DIR = r'C:\Users\fbertil\CRC_slides_restain\Slide 3\K10985-13\stain0\CP_masks'
OUTPUT_DIR = r'C:\Users\fbertil\CRC_slides_restain\Slide 3\K10985-13\features_stain0'

MARKERS_CONFIG = {
    
            'DAPI':   {'channel': 0, 
                      'compartment': 'nuclear', 
                      'features': {'firstorder': ['Mean']},
                      'params':{}},  


           }

X_SPACING = 1 # Convertion from pixel to physical unit 
Y_SPACING = 1 # Convertion from pixel to physical unit 
DELTA = 5 # Crop parameter
CYTO_OFFSET = 5 # Cytoplasmic mask parameter
IN_OFFSET = -1 # Perinuclear mask parameter
OUT_OFFSET = 2 # Perinuclear mask parameter
FEATURES = ['shape','intensity']

image_paths = glob.glob(DIRECTORY + '/*.tif')

# Time profiler

In [16]:
# Time profiler

%load_ext line_profiler
%lprun -f compute_single_cell_features compute_single_cell_features(img_path, MASK_DIR, OUTPUT_DIR, MARKERS_CONFIG, FEATURES, remove_border=False, out_format='h5', cyto_offset=CYTO_OFFSET)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# Batch processing

In [7]:
from joblib import Parallel, delayed

Parallel(n_jobs=10)(delayed(compute_single_cell_features(img_path,
                                                         MASK_DIR,
                                                         OUTPUT_DIR,
                                                         MARKERS_CONFIG,
                                                         FEATURES,
                                                         remove_border=False,
                                                         out_format='h5',
                                                         cyto_offset=CYTO_OFFSET
                                                         ))(img_path) for img_path in image_paths)

TypeError: 'NoneType' object is not callable